In [ ]:
import numpy as np
import tensorflow as tf

from deeprte.model.tf import input_pipeline

In [ ]:
split = input_pipeline.Split.TRAIN

ds = input_pipeline.load(
    split=split,
    split_ratio=0.8,
    is_training=True,
    batch_sizes=[2, 3],
    collocation_sizes=[120, 30],
)

In [ ]:
for i in range(3):
    print(tf.nest.map_structure(lambda x: x.shape, next(iter(ds))))